# Complementaria Repaso Quiz 3 - Miércoles 3:30

In [2]:
import numpy as np
import pandas as pd
from jmarkov.mdp.dtmdp import dtmdp

In [3]:
# Matrices de transición (orden de estados: F, N, H)

P_S = np.array([
    [0.60, 0.30, 0.10],
    [0.10, 0.60, 0.30],
    [0.00, 0.30, 0.70]
])

P_B = np.array([
    [0.70, 0.25, 0.05],
    [0.20, 0.65, 0.15],
    [0.10, 0.60, 0.30]
])

P_V = np.array([
    [0.85, 0.15, 0.00],
    [0.40, 0.55, 0.05],
    [0.20, 0.60, 0.20]
])

P_R = np.array([
    [0.90, 0.10, 0.00],
    [0.50, 0.45, 0.05],
    [0.30, 0.60, 0.10]
])

# Costos térmicos por estado (USD por hora)
costos_termicos = {
    'F': 60,
    'N': 100,
    'H': 250
}

# Costos por acción (USD por hora)
costos_accion = {
    'S': 0,    # Sin ajuste
    'B': 40,   # Balanceo de carga
    'V': 70,   # Ventilación extra
    'R': 90    # Reducción de capacidad
}


## Parte A: Implementación del MDP

In [20]:
# estados

estados = np.array(["F","N","H"], dtype=str)

# acciones

acciones = np.array(["S","B","V","R"], dtype=str)

# Matrices


matrices = {
    "S" : P_S,
    "B" : P_B,
    "V" : P_V,
    "R" : P_R
    }

# retornos (minimar costo)

retornos = np.zeros((len(estados),len(acciones)))

for i, s in enumerate(estados):
    for j, a in enumerate(acciones):
        retornos[i,j] = costos_accion[a] + costos_termicos[s]

minimize = True

retornos_df = pd.DataFrame(retornos, index = estados, columns = acciones)

# MDP

beta = 0.95

mdp = dtmdp(estados, acciones, matrices, retornos, beta)

# Valores optimos y politica optima

value_functions, optimal_policy = mdp.solve(0, minimize=minimize, method="value_iteration")

print(value_functions)
print(optimal_policy)

# Valor esperado de la politica optima

mdp.expected_policy_value(value_functions, optimal_policy)

# Matriz de transicion

matrizPO = mdp.policy_transition_matrix(optimal_policy)

matrizPO_df = pd.DataFrame(matrizPO, index=estados, columns = estados)
matrizPO_df


[2509.73696485 2634.34434529 2825.25006827]
{np.str_('F'): np.str_('S'), np.str_('N'): np.str_('V'), np.str_('H'): np.str_('R')}


,F,N,H
F,0.6,0.30,0.10
N,0.4,0.55,0.05
H,0.3,0.60,0.10


## Parte B: Simulación de Monte Carlo

In [25]:
escenarios = 1000 # escenarios
horizon = 24 # horas
estado_inicial = "N"

np.random.seed(0)

Y = np.zeros((horizon, escenarios))

indicador_evento = np.zeros(escenarios)

for j in range(escenarios):

    estado = estado_inicial
    visito_evento = 0

    for t in range(horizon):
        accion = str(optimal_policy[estado])

        Y[t,j] = retornos_df.loc[estado, accion]

        if t < 8 and estado == "F":
            visito_evento = 1

        estado = np.random.choice(estados, p=matrizPO_df.loc[estado,:].values)

    indicador_evento[j] = visito_evento


In [31]:
# Cual es el costo acumulado esperado durante las 24 horas - Caso 2

respuesta_1 = Y.sum(axis=0).mean()

# ¿Cu ́al es la probabilidad estimada de que la sala visite el estado Fr ́ıo (F) al menos una vez en las primeras 8 horas? - Caso 5

respuesta_2 = indicador_evento.mean()*100

print(respuesta_1)
print(f"{respuesta_2}%")



3139.5
96.7%
